<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 2.0 NLP Projects with NVIDIA NeMo

<img style="float: right;" src="images/nemo/nemo-app-stack.png" width=350>


In this notebook, you'll learn the basics of setting up an NLP project with [NVIDIA NeMo](https://developer.nvidia.com/nvidia-nemo).  You'll learn what components are needed, where to find them, and how to instantiate them.  

**[2.1 Programming Model](#2.1-Programming-Model)**<br>
**[2.2 Create Neural Modules (Step 1)](#2.2-Create-Neural-Modules-(Step-1))**<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2.1 Data Layer and Tokenizer](#2.2.1-Data-Layer-and-Tokenizer)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2.2 Language Model and Classifier](#2.2.2-Language-Model-and-Classifier)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2.3 Loss Function](#2.2.3-Loss-Function)<br>
**[2.3 Create Neural Graph (Step 2)](#2.3-Create-Neural-Graph-(Step-2))**<br>
**[2.4 Start an Action (Step 3)](#2.4-Start-an-Action-(Step-3))**<br>

NeMo is an open source, high-performance toolkit for building conversational AI applications using a PyTorch backend.  NeMo provides a level of abstraction beyond Keras or PyTorch, that makes it possible to easily compose complex neural network architectures using reusable components called _neural modules_ (hence the name NeMo). NeMo includes capabilities to scale training to multi-GPU systems and multi-node clusters as well, and provides a straightforward path to model deployment for production real-time inference. 




# 2.1 Programming Model
NeMo includes library collections for all aspects of the conversational AI pipeline: speech recognition (`nemo_asr`), natural language (`nemo_nlp`), and speech synthesis (`nemo_tts`).  For our projects, we'll use the `nemo-nlp` and some core libraries.

Building an application using NeMo APIs consists of three basic steps:
1. **Create neural modules** after instantiating the NeuralModuleFactory:<br>
A _NeuralModule_ is an abstraction between a network layer and a full neural network, for example: encoder, decoder, language model, acoustic model, etc.  
1. **Create the neural graph** of tensors connecting the neural modules together:<br>
Define all the inputs, outputs, and connections you need to form a pipeline.
1. **Start an "action"** such as `train` for training a network or `infer` obtain a result from a network:<br>
We can also define _callbacks_ for evaluation, logging and performance monitoring.


In [3]:
# Start with the nemo and nlp imports (Ignore the torchaudio error if it appears).
# Import "inspect" so we can look at method signatures.
import nemo
import nemo.collections.nlp as nemo_nlp
import inspect

# 2.2 Create Neural Modules (Step 1)

Before creating any neural modules, we must instantiate the `NeuralModuleFactory` object.  There are a number of settings that we'll use in the projects to specify directories for intermediate information such as checkpoints and logs.  This is how to instantiate the factory.

In [4]:
# Instantiate the NeuralModuleFactory before creating neural modules.
nf = nemo.core.NeuralModuleFactory()

Neural Modules can be conceptually classified into four potentially overlapping categories:

* Data Layers - modules that perform extract, transform, load, and feed of the data
* Trainable Modules - modules that contain trainable weights
* Non Trainable Modules - non-trainable module, for example, table lookup, data augmentation, greedy decoder, etc. 
* Loss Modules - modules that compute loss functions

For text classification and NER, we will need a data layer, a trainable language model, a trainable classifier, and a loss module (shown as boxes in the diagram below).  

<img src="images/nemo/nm-pipe.png" width=800>
     
It is possible to build your own neural modules, but there are plenty available for us to use already.  

## 2.2.1 Data Layer and Tokenizer
Run the following cell to see a list of available NLP data layer neural modules.

In [5]:
# List the callable data layer objects.
[method_name for method_name in dir(nemo_nlp.nm.data_layers)
                  if callable(getattr(nemo_nlp.nm.data_layers, method_name)) 
                     and method_name[0] not in ['_']
]

['BertInferDataLayer',
 'BertJointIntentSlotDataLayer',
 'BertJointIntentSlotInferDataLayer',
 'BertPretrainingDataLayer',
 'BertPretrainingPreprocessedDataLayer',
 'BertQuestionAnsweringDataLayer',
 'BertTextClassificationDataLayer',
 'BertTokenClassificationDataLayer',
 'BertTokenClassificationInferDataLayer',
 'GlueClassificationDataLayer',
 'GlueRegressionDataLayer',
 'LanguageModelingDataLayer',
 'LaserTaggerDataLayer',
 'MultiWOZDataLayer',
 'PunctuationCapitalizationDataLayer',
 'SGDDataLayer',
 'TextDataLayer',
 'TranslationDataLayer']

For text classification project, we'll use the `BertTextClassificationDataLayer` neural module, while for NER we'll use `BertTokenClassificationDataLayer`.  Take a look at the class signatures below. 

In [6]:
inspect.signature(nemo_nlp.nm.data_layers.BertTextClassificationDataLayer)

<Signature (input_file, tokenizer, max_seq_length, num_samples=-1, shuffle=False, batch_size=64, use_cache=False, dataset_type=<class 'nemo.collections.nlp.data.datasets.text_classification.text_classification_dataset.BertTextClassificationDataset'>)>

In [7]:
inspect.signature(nemo_nlp.nm.data_layers.BertTokenClassificationDataLayer)

<Signature (text_file, label_file, tokenizer, max_seq_length, pad_label='O', label_ids=None, num_samples=-1, shuffle=False, batch_size=64, ignore_extra_tokens=False, ignore_start_end=False, use_cache=False, dataset_type=<class 'nemo.collections.nlp.data.datasets.token_classification_dataset.BertTokenClassificationDataset'>)>

Note that it has a few differences, such as the imported `text_file` and `label_file` parameters needed for NER, rather than the single `input_file` used for the text classification task.

In both cases, we need to also provide the `tokenizer` and `max_seq_length` parameters.  For the maximum sequence length, we have a tradeoff here between providing enough information for the system to employ useful attention, while maintaining efficiency of memory and processing time.  The maximum size for BERT is 512. Depending on the particular task and constraints, we might typically choose 32, 64, 128, 256, or 512.

### Tokenizer
A _tokenizer_ segments text and documents into a given type of token, which may be sentences, sentence pieces, words, word pieces, or even characters.  Take a look at the choices currently available within the NeMo library:

In [8]:
# List the callable tokenizer objects.
[method_name for method_name in dir(nemo_nlp.data.tokenizers)
                  if callable(getattr(nemo_nlp.data.tokenizers, method_name)) 
                     and method_name[0] not in ['_']
]

['CharTokenizer',
 'NemoBertTokenizer',
 'NemoGPT2Tokenizer',
 'SentencePieceTokenizer',
 'WordTokenizer',
 'YouTokenToMeTokenizer',
 'get_bert_special_tokens',
 'get_tokenizer']

Since we'll be starting with pretrained BERT models and fine-tuning from there, our tokenizer must use the one that the BERT model expects, which happens to be the `NemoBertTokenizer`.  In addition to breaking apart the input text, the tokenizer converts each token to an _id_ associated with a vocabulary file.  This vocabulary can either be provided by the user, or must be calculated when the tokenizer is specified, which may take a bit of time.  

The details of the tokenizer instantiation can be abstracted somewhat by using the `get_tokenizer` helper.  This is a little easier in practice since it abstracts some details related to exactly which BERT or Megatron pretrained model we choose.  For our example, we'll use `bert-base-uncased`.

In [9]:
tokenizer = nemo.collections.nlp.data.tokenizers.get_tokenizer(tokenizer_name='nemobert', 
                                                               pretrained_model_name='bert-base-uncased')

[NeMo I 2021-06-08 15:51:29 bert_tokenizer:78] Deriving bert model type from pretrained model name.


The BERT tokenizer uses _wordpiece_ tokenization, and since we have specified an uncased model, we would expect the tokenizer to turn all the words to lower case as well.  Run the following cell to see how the tokenizer breaks apart a sentence with a wordpiece algorithm, and encodes it.

In [10]:
# Show tokens from an input
test_sentence = 'How about this stupendous sentence?'
tokenized = tokenizer.text_to_tokens(test_sentence)
encoded = tokenizer.text_to_ids(test_sentence)
print('{}\n{}\n{}'.format(test_sentence, tokenized, encoded))

How about this stupendous sentence?
['how', 'about', 'this', 'stu', '##pen', '##dou', '##s', 'sentence', '?']
[2129, 2055, 2023, 24646, 11837, 26797, 2015, 6251, 1029]


We can now instantiate a nominal data layer neural module.

In [12]:
dl = nemo_nlp.nm.data_layers.BertTextClassificationDataLayer(input_file='data/NCBI_tc-3/dev.tsv', 
                                                             tokenizer=tokenizer, 
                                                             max_seq_length=128)

[NeMo I 2021-06-08 15:53:01 text_classification_dataset:82] data/NCBI_tc-3/dev.tsv: 100
[NeMo I 2021-06-08 15:53:03 data_preprocessing:250] Min: 120 |                  Max: 506 |                  Mean: 307.0 |                  Median: 301.5
[NeMo I 2021-06-08 15:53:03 data_preprocessing:252] 75 percentile: 363.0
[NeMo I 2021-06-08 15:53:03 data_preprocessing:253] 99 percentile: 503.03
[NeMo I 2021-06-08 15:53:03 text_classification_dataset:118] 99 out of 100                         sentences with more than 128 subtokens.
[NeMo I 2021-06-08 15:53:03 text_classification_dataset:175] *** Example ***
[NeMo I 2021-06-08 15:53:03 text_classification_dataset:176] example_index: 0
[NeMo I 2021-06-08 15:53:03 text_classification_dataset:177] subtokens: [CLS] remained elusive . we now show that br ##ca ##1 en ##codes a 190 - k ##d protein with sequence homo ##logy and bio ##chemical analogy to the gran ##in protein family . interesting ##ly , br ##ca ##2 also includes a motif similar to the gran

## 2.2.2 Language Model and Classifier
Run the following cell to see a list of available NLP trainable neural modules.

In [13]:
# List the callable trainable neural module objects
[method_name for method_name in dir(nemo_nlp.nm.trainables)
                  if callable(getattr(nemo_nlp.nm.trainables, method_name)) 
                     and method_name[0] not in ['_']
]

['Albert',
 'BERT',
 'BeamSearchTranslatorNM',
 'BertTokenClassifier',
 'EncoderRNN',
 'GreedyLanguageGeneratorNM',
 'JointIntentSlotClassifier',
 'PunctCapitTokenClassifier',
 'Roberta',
 'SGDDecoderNM',
 'SGDEncoderNM',
 'SequenceClassifier',
 'SequenceRegression',
 'TRADEGenerator',
 'TokenClassifier',
 'TransformerDecoderNM',
 'TransformerEncoderNM',
 'get_huggingface_lm_model',
 'get_huggingface_lm_models_list',
 'get_megatron_checkpoint',
 'get_megatron_config_file',
 'get_megatron_lm_models_list',
 'get_megatron_vocab_file',
 'get_pretrained_lm_model',
 'get_pretrained_lm_models_list']

### Language Model
We'll use the `get_pretrained_lm_model` method to specify a language model.  We can see a list of available models for us to try.  

In [14]:
nemo_nlp.nm.trainables.get_pretrained_lm_models_list()

['megatron-bert-345m-uncased',
 'megatron-bert-345m-cased',
 'megatron-bert-uncased',
 'megatron-bert-cased',
 'bert-base-uncased',
 'bert-large-uncased',
 'bert-base-cased',
 'bert-large-cased',
 'bert-base-multilingual-uncased',
 'bert-base-multilingual-cased',
 'bert-base-chinese',
 'bert-base-german-cased',
 'bert-large-uncased-whole-word-masking',
 'bert-large-cased-whole-word-masking',
 'bert-large-uncased-whole-word-masking-finetuned-squad',
 'bert-large-cased-whole-word-masking-finetuned-squad',
 'bert-base-cased-finetuned-mrpc',
 'bert-base-german-dbmdz-cased',
 'bert-base-german-dbmdz-uncased',
 'cl-tohoku/bert-base-japanese',
 'cl-tohoku/bert-base-japanese-whole-word-masking',
 'cl-tohoku/bert-base-japanese-char',
 'cl-tohoku/bert-base-japanese-char-whole-word-masking',
 'TurkuNLP/bert-base-finnish-cased-v1',
 'TurkuNLP/bert-base-finnish-uncased-v1',
 'wietsedv/bert-base-dutch-cased',
 'roberta-base',
 'roberta-large',
 'roberta-large-mnli',
 'distilroberta-base',
 'roberta-

It's easy to instantiate this type of neural module.  Recall that we already chose 'bert-base-uncased' for the tokenizer. The language model chosen here should match.

In [15]:
# Instantiate the language model.
lm = nemo_nlp.nm.trainables.get_pretrained_lm_model('bert-base-uncased')

### Classification Model
For text classification we'll need the `SequenceClassifier` trainable neural module, while the NER project will require the similar `TokenClassifier` neural module.  Take a look at the signatures below:

In [16]:
inspect.signature(nemo_nlp.nm.trainables.SequenceClassifier)

<Signature (hidden_size, num_classes, num_layers=2, activation='relu', log_softmax=True, dropout=0.0, use_transformer_pretrained=True)>

In [17]:
inspect.signature(nemo_nlp.nm.trainables.TokenClassifier)

<Signature (hidden_size, num_classes, name=None, num_layers=2, activation='relu', log_softmax=True, dropout=0.0, use_transformer_pretrained=True)>

In both cases, we must provide the `hidden_size`, which we can pull from the language model, and the number of label categories in `num_classes`.  We may wish to alter the other default values as well.

In [18]:
# Instantiate the classification neural module
cl = nemo_nlp.nm.trainables.SequenceClassifier(hidden_size=lm.hidden_size, num_classes=3)

## 2.2.3 Loss Function
Finally, we define the loss function neural module.  For both of our projects, we will use `CrossEntropyLossNM`.  Here's a list of what's available:

In [19]:
# List the callable loss common neural module objects
[method_name for method_name in dir(nemo.backends.pytorch.common.losses)
                  if callable(getattr(nemo.backends.pytorch.common.losses, method_name)) 
                     and method_name[0] not in ['_']
]

['BCEWithLogitsLossNM',
 'CrossEntropyLossNM',
 'LabelsType',
 'LogitsType',
 'LossAggregatorNM',
 'LossNM',
 'LossType',
 'MSELoss',
 'MaskType',
 'NeuralType',
 'RegressionValuesType',
 'SequenceLoss',
 'add_port_docs']

In [20]:
inspect.signature(nemo.backends.pytorch.common.losses.CrossEntropyLossNM)

<Signature (logits_ndim=2, weight=None, reduction='mean')>

In [21]:
# Instantiate the loss function neural module
lf = nemo.backends.pytorch.common.losses.CrossEntropyLossNM()

# 2.3 Create Neural Graph

Now that there are four modules in place, defining the graph is just a matter of connecting each of the modules.  We'll define the output of each as the input of the next in the pipeline.

To set up the data link, first get the outputs from the data layer. 

<img src="images/nemo/data-tensors.png" width=250>

In [22]:
input_data, token_types, attn_mask, labels = dl()

Next, determine the embeddings, or outputs from the language model, based on the data inputs provided by the data layer.

<img src="images/nemo/embedding-tensors.png" width=250>

In [23]:
embeddings = lm(input_ids=input_data, token_type_ids=token_types, attention_mask=attn_mask)

In a similar fashion, define the logits (the raw, non-normalized prediction tensors) generated by the classifier.  The loss function outputs are dependent upon both the logits and the labels available from the data layer.

<img src="images/nemo/logits-tensors.png" width=320>

In [24]:
logits = cl(hidden_states=embeddings)
loss=lf(logits=logits, labels=labels)

Great!  Now that a graph is defined and ready for action. 

# 2.4 Start an Action

The actions of training and inference are run from the `NeuralModuleFactory` that you instantiated at the start of the exercise.  Though we don't need to actually execute the actions in this notebook, take a look at the method signatures to see what kind of information we'll need to provide.

In [25]:
inspect.signature(nemo.core.NeuralModuleFactory.train)

<Signature (self, tensors_to_optimize=None, training_graph=None, optimizer=None, optimization_params=None, callbacks:Union[List[Union[nemo.core.deprecated_callbacks.ActionCallback, nemo.core.callbacks.NeMoCallback]], NoneType]=None, lr_policy=None, batches_per_step=None, stop_on_nan_loss=False, synced_batchnorm=False, synced_batchnorm_groupsize=0, gradient_predivide=False, amp_max_loss_scale=16777216.0, reset=False)>

In [26]:
inspect.signature(nemo.core.NeuralModuleFactory.infer)

<Signature (self, tensors:List[nemo.core.neural_types.neural_type.NmTensor], checkpoint_dir=None, ckpt_pattern='', verbose=True, cache=False, use_cache=False, offload_to_cpu=True, modules_to_restore=None)>

To complete the setup for training in the project notebooks, you'll need to specify some functions for optimization, learning rate policy, and callbacks. The callbacks are invoked during the processing for purposes or recording intermediate parameters.  This is useful for saving checkpoints and monitoring results. The signatures can be viewed below, and actual use examples are provided in the project notebooks.

In [ ]:
inspect.signature(nemo.core.SimpleLogger)

In [ ]:
inspect.signature(nemo.core.TensorboardLogger)

In [ ]:
inspect.signature(nemo.core.EvaluatorCallback)

In [ ]:
inspect.signature(nemo.core.CheckpointCallback)

<h2 style="color:green;">Congratulations!</h2>

You've explored the NeMo API and learned:
* The three parts of a project in NeMo are: neural modules, neural graphs, and an action
* Neural modules we need for these projects are data layers, language models, classifiers, and loss functions
* Neural graphs are created by defining the inputs and outputs between the neural modules
* Actions are `train` and `infer`

You are ready to build the text classification project.<br>

Move on to [3.0 Build a 3-class Text Classifier](030_TextClassification.ipynb).

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>